In [1]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D,Dropout,BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
%matplotlib inline

In [ ]:
def weightInit(shape,dtype=None):
    """Initialize weights"""
    return rng.normal(loc = 0.0, scale = 1e-2, size = shape)
#//TODO: figure out how to initialize layer biases in keras.
def biasInit(shape,dtype=None):
    """Initialize bias"""
    return rng.normal(loc = 0.5, scale = 1e-2, size = shape)
input_shape = (300, 300, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)

model = Sequential()
model.add(Conv2D(64,(6,6),activation='relu',input_shape=input_shape,kernel_initializer=weightInit,kernel_regularizer=l2(2e-4)))
model.add(MaxPooling2D())
model.add(Conv2D(128,(4,4),activation='relu',kernel_regularizer=l2(2e-4),kernel_initializer=weightInit,bias_initializer=biasInit))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.1))
model.add(Conv2D(128,(3,3),activation='relu',kernel_initializer=weightInit,kernel_regularizer=l2(2e-4),bias_initializer=biasInit))
# model.add(MaxPooling2D())
# model.add(Conv2D(256,(2,2),activation='relu',kernel_initializer=weightInit,kernel_regularizer=l2(2e-4),bias_initializer=biasInit))
# model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.6))
model.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=weightInit,bias_initializer=biasInit))

#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = model(left_input)
encoded_r = model(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid',bias_initializer=biasInit)(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
optimizer = Adam(0.00006)
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

In [ ]:
siamese_net.summary()

In [ ]:
%run DatasetPrepare.ipynb

In [ ]:
createLabelsTargets = CreateLabelsTargets("train_HB.csv")
createLabelsTargets.load_csv_and_treat() 
(X_train, X_test, y_train, y_test) = createLabelsTargets.create_labels_and_targets()

In [ ]:
imagesPath= os.getcwd()+'/modelHB_imgs/train/'
# Creating training data generator
dataTrainLoader = DataLoader(imagesPath,X_train, y_train)


In [ ]:
#Training loop
evaluate_every = 50 # interval for evaluating on one-shot tasks
loss_every= 50 # interval for printing loss (iterations)
batch_size = 16
epochs = 9000
N_way = 5 # how many classes for testing one-shot tasks
n_val = 100 #number of one-shot tasks to validate on
lossHistory = []
val_accHistory = []
print("training")
for i in range(1, epochs):
    (inputs,targets)=dataTrainLoader.getBatch(batch_size)
    (loss)=siamese_net.train_on_batch(inputs,targets)
    lossHistory.append(loss)
    if i % evaluate_every == 0:
        val_acc = dataTrainLoader.test_oneshot(siamese_net,N_way,n_val,X_test,y_test,verbose=True)
        val_accHistory.append(val_acc)
    if i % loss_every == 0:
        print("iteration {}, loss: {:.2f}, val_acc: {:.2f}".format(i,loss,val_acc))


In [ ]:
plt.plot(lossHistory)
plt.show()
plt.plot(val_accHistory)
plt.show()

In [19]:
#usar evaluate para ver loss
# passar no teste x_train e X_test, para ver se é overfitting
#suspeita de overfitting
